In [1]:
from WebTable import findTables, pullTable
from bs4 import BeautifulSoup as BS
from bs4 import Comment
from gazpacho import get, Soup
import pandas as pd
import re
from tqdm import tnrange, tqdm_notebook
from time import sleep
import warnings
warnings.filterwarnings('ignore')

In [2]:
team_list = ['crd', 'atl', 'rav', 'buf', 'car', 'chi', 'cin', 'cle', 'dal', 'den', 'det','gnb', 
'htx', 'clt', 'jax', 'kan', 'mia', 'min', 'nwe', 'nor', 'nyg', 'nyj', 'rai', 'phi', 'pit', 
'sdg', 'sfo', 'sea', 'ram', 'tam', 'oti', 'was']

In [3]:
def rename_Abv2Full(team_s):
    try:
        if team_s == 'crd' :
            ts = 'Arizona Cardinals'
        elif team_s == 'atl':
            ts = 'Atlanta Falcons'
        elif team_s == 'rav':
            ts = 'Baltimore Ravens'
        elif team_s == 'buf':
            ts = 'Buffalo Bills'
        elif team_s == 'car':
            ts = 'Carolina Panthers'
        elif team_s == 'chi':
            ts = 'Chicago Bears'
        elif team_s == 'cin':
            ts = 'Cincinnati Bengals'
        elif team_s == 'cle':
            ts = 'Cleveland Browns'
        elif team_s == 'dal':
            ts = 'Dallas Cowboys'
        elif team_s == 'den':
            ts = 'Denver Broncos'
        elif team_s == 'det':
            ts = 'Detroit Lions'
        elif team_s == 'gnb':
            ts = 'Green Bay Packers'
        elif team_s == 'htx':
            ts = 'Houston Texans'
        elif team_s == 'clt':
            ts = 'Indianapolis Colts'
        elif team_s == 'jax':
            ts = 'Jacksonville Jaguars'
        elif team_s == 'kan':
            ts = 'Kansas City Chiefs'
        elif team_s == 'mia':
            ts = 'Miami Dolphins'
        elif team_s == 'min':
            ts = 'Minnesota Vikings'
        elif team_s == 'nwe':
            ts = 'New England Patriots'
        elif team_s == 'nor':
            ts = 'New Orleans Saints'
        elif team_s == 'nyg':
            ts = 'New York Giants'
        elif team_s == 'nyj':
            ts = 'New York Jets'
        elif team_s == 'rai':
            ts = 'Oakland Raiders'
        elif team_s == 'phi':
            ts = 'Philadelphia Eagles'
        elif team_s == 'pit':
            ts = 'Pittsburgh Steelers'
        elif team_s == 'sdg':
            ts = 'San Diego Chargers'
        elif team_s == 'sfo':
            ts = 'San Francisco 49ers'
        elif team_s == 'sea':
            ts = 'Seattle Seahawks'
        elif team_s == 'ram':
            ts = 'St. Louis Rams'
        elif team_s == 'tam':
            ts = 'Tampa Bay Buccaneers'
        elif team_s == 'oti':
            ts = 'Tennessee Titans'
        elif team_s == 'was':
            ts = 'Washington Redskins'
    except NameError:
        print('Invalid team name')
    return ts

In [4]:
def renameFull2Abv(team_s):
    try:
        if team_s == 'Arizona Cardinals':
            ts = 'crd'
        elif team_s == 'Atlanta Falcons':
            ts = 'atl'
        elif team_s == 'Baltimore Ravens':
            ts = 'rav'
        elif team_s == 'Buffalo Bills':
            ts = 'buf'
        elif team_s == 'Carolina Panthers':
            ts = 'car'
        elif team_s == 'Chicago Bears':
            ts = 'chi'
        elif team_s == 'Cincinnati Bengals':
            ts = 'cin'
        elif team_s == 'Cleveland Browns':
            ts = 'cle'
        elif team_s == 'Dallas Cowboys':
            ts = 'dal'
        elif team_s == 'Denver Broncos':
            ts = 'den'
        elif team_s == 'Detroit Lions':
            ts = 'det'
        elif team_s == 'Green Bay Packers':
            ts = 'gnb'
        elif team_s == 'Houston Texans':
            ts = 'htx'
        elif team_s == 'Indianapolis Colts':
            ts = 'clt'
        elif team_s == 'Jacksonville Jaguars':
            ts = 'jax'
        elif team_s == 'Kansas City Chiefs':
            ts = 'kan'
        elif team_s == 'Miami Dolphins':
            ts = 'mia'
        elif team_s == 'Minnesota Vikings':
            ts = 'min'
        elif team_s == 'New England Patriots':
            ts = 'nwe'
        elif team_s == 'New Orleans Saints':
            ts = 'nor'
        elif team_s == 'New York Giants':
            ts = 'nyg'
        elif team_s == 'New York Jets':
            ts = 'nyj'
        elif team_s == 'Oakland Raiders':
            ts = 'rai'
        elif team_s == 'Philadelphia Eagles':
            ts = 'phi'
        elif team_s == 'Pittsburgh Steelers':
            ts = 'pit'
        elif team_s == 'San Diego Chargers':
            ts = 'sdg'
        elif team_s == 'San Francisco 49ers':
            ts = 'sfo'
        elif team_s == 'Seattle Seahawks':
            ts = 'sea'
        elif team_s == 'St. Louis Rams':
            ts = 'ram'
        elif team_s == 'Tampa Bay Buccaneers':
            ts = 'tam'
        elif team_s == 'Tennessee Titans':
            ts = 'oti'
        elif team_s == 'Washington Redskins':
            ts = 'was'
    except NameError:
        print('Invalid team name')
    return ts

In [5]:
# col_name = ['Week','Day','Date','Unnamed: 3_level_1','Unnamed: 4_level_1','Unnamed: 5_level_1','OT',
#             'Rec','Unnamed: 8_level_1','Opp','Tm','Opp','1stD','TotYd','PassY','RushY','TO','1stD',
#             'TotYd','PassY','RushY','TO','Offense','Defense','Sp. Tms','Name','Year']
# col_name2 = ['Week','Day','Date','Time','Boxscore','W/L','OT','Rec','Home/Away','Opp',
#              'Score_Home','Opp_Score','1stD','TotYd','PassY','RushY','TO','Opp_1stD','Opp_TotYd','Opp_PassY',
#              'Opp_RushY','Opp_TO','Offense','Defense','Sp. Tms','Name','Year']

In [6]:
def urlSplit(inputlist):
    ''' Used to slice the boxscore link to get and sort link and date
    '''
    urlList = []
    for item in inputlist:
        sort = item.split('/')[-1][:9]
        urlList.append(sort)
    return sorted(urlList, reverse=False)

def Nanreplace(x):
    if pd.isnull(x['relative_humidity']):
        return x['relative humidity']
    else:
        return x['relative_humidity']

def boxscore(url):
    '''returns the link to the boxscore and the date the game was played'''
    html = get(url)
    soup = Soup(html)
    boxlinks = str(soup.find('a', {'href': 'boxscore'})).split('"')                    # Finds all a tags with a href link and splits on "
    links_list = [x for x in boxlinks if x.startswith('/boxscore')][1:-3]              # creates a list of all the links that starts with /boxscore
    links_list = ['https://www.pro-football-reference.com' + x for x in links_list]    # creates the complete url to the boxscore links
    df = pd.DataFrame(links_list, columns =['Links'])                                  # creates a dataframe containing all of the url links just created
    df['Date'] = urlSplit(links_list)                                                  # add a new column that uses urlSplit to get an idea of the date
    return df

def get_boxscore_stats(url):
    '''reutns a dataframe that contains home and away boxscores
    use the link from the boxscore link generated from the master
    dataframe
    '''
    boxstats = pullTable(url,'team_stats')                                             # Pull box table from boxscore url
    boxstats.columns = ['Stats', 'Away', 'Home']                                       # rename column names
    homebox = boxstats[['Stats','Home']]                                               # create a list for home and away teams
    awaybox = boxstats[['Stats','Away']]
    homebox = homebox.transpose()                                                      # Transpose the tables
    awaybox = awaybox.transpose()
    home_colnames = ['Home_' + x for x in homebox.iloc[0]]                             # add Home or Away to column names
    away_colnames = ['Away_' + x for x in awaybox.iloc[0]]
    zip_Homeboxscore = list(zip(home_colnames, homebox.iloc[1]))                       # zip list of columns and data together
    zip_Awayboxscore = list(zip(away_colnames, awaybox.iloc[1]))
    boxscorelist = zip_Homeboxscore + zip_Awayboxscore                                 # combine lists together
    box_col = [col[0].replace(' ','_').replace('.','') for col in boxscorelist]        # Replace spaces with _ and remove .
    box_stats = pd.Series([col[1] for col in boxscorelist], index=box_col)             
    boxdf= pd.DataFrame(columns=box_col)
    boxdf = boxdf.append(box_stats, ignore_index=True)
    boxdf['GetBoxScoreURL'] = url
    return boxdf


def getGameInfo(url):
    '''Gets table from Game Info table that is within a HTML comment
    Will return a list of lists were [n][0] = column name and [n][1] is the infomation
    [('Won_Toss', 'Cardinals (deferred)'),
     ('Roof', 'retractable roof (closed)'),
     ('Surface', 'grass'),
     ('Duration', '2:56'),
     ('Attendance', ' 60,104 '),
     ('Vegas Line', 'Arizona Cardinals -2.5'),
     ('Over/Under', '46.0(over) ')
    '''
    html = get(url)
    soup = BS(html, 'html.parser')
    comments = soup.find_all(string=lambda text: isinstance(text, Comment))                        # retrieve all the comments from the HTML
    game_comments = []

    for line in comments:
        if '<div class="table_outer_container">' in line:                                          # loop throught the table i want
            game_comments.append(re.sub('<[^>]+>', ' ', line))                                     # use regex to replace everything between the <> tags and append to game comments
            if '</table>' in line:                                                                 # break out of the for loop once it hits the ending table tag 
                break
    game_info = [x.split('\n') for x in game_comments]                                             # split on newlines to create a clean list
    game_info = game_info[0][4:12]                                                                 # more cleaning, i only want these 8 items
    game_info = [x.replace('Won Toss','Won_Toss') if 'Won Toss' in x else x for x in game_info]    # replace the space in the Won Toss item, cleaning/prep
    game_info                                                                               
    for count, row in enumerate(game_info):
        if count == 5 and 'Weather' in row:                                                        
            game_info.pop(count)
        if count == 7:
            game_info.pop(count)
    game_info = [x.replace('  ','|') for x in game_info]
    #split on | and setting max split to 2 and then removing the empty list items 
    game_titles = [x.split('|',2)[1] for x in game_info]
    game_data = [x.split('|',2)[2].replace('|','') for x in game_info]
    game = pd.DataFrame([game_data], columns = game_titles)
    game['VegasLink'] = url
    return game

def get_master_df(teamList, startyear, endyear):
    endyear = endyear + 1
#     years = [x for x in range(startyear,endyear)]
    masterdf = pd.DataFrame()
    for x in tnrange(startyear,endyear, desc='YEAR'):
        nflDF = pd.DataFrame()
        for team in tqdm_notebook((team_list), desc='TEAMS'):
            year = x
            base = 'https://www.pro-football-reference.com'
            url = base + '/teams/' + team + '/' + str(year) + '.htm'
            html = get(url)
            soup = Soup(html)
            df = pd.read_html(str(soup.find('table')[1]))[0]
            df = df[df['Unnamed: 9_level_0','Opp'] != 'Bye Week']
            df = df[df['Unnamed: 9_level_0','Opp'] != 'Playoffs']
            df['Team','Name'] = rename_Abv2Full(team)
            df['Team','Year'] = year
            top_level = df.columns.get_level_values(0).astype(str)
            lower_level = df.columns.get_level_values(1).astype(str)
            df.columns = top_level + '_' + lower_level
            df.columns = [x.replace("', '","_").replace('Offense','Home').replace('Defense','Away') for x in df.columns]
            df.rename(columns = {'Unnamed: 0_level_0_Week':'Week','Unnamed: 1_level_0_Day':'Day','Unnamed: 2_level_0_Date':'Date',
                                      'Unnamed: 3_level_0_Unnamed: 3_level_1':'Time_ET','Unnamed: 4_level_0_Unnamed: 4_level_1':'BOXSCORE',
                                      'Unnamed: 5_level_0_Unnamed: 5_level_1':'W_L','Unnamed: 6_level_0_OT':'OT','Unnamed: 7_level_0_Rec':'Home_Record',
                                      'Unnamed: 8_level_0_Unnamed: 8_level_1':'HomeOrAway','Unnamed: 9_level_0_Opp':'Opp','Score_Tm':'Home_Score',
                                      'Score_Opp':'Away_Score'},inplace=True)
            df.drop(['Home_1stD','Home_PassY','Home_RushY','Home_TO','Away_1stD','Away_PassY','Away_TO'], axis=1, inplace=True)
            df = df.reset_index(drop=True)
            boxscorelink = boxscore(url)
            mergedf =  df.merge(boxscorelink, left_index=True, right_index=True)
            gameinfo =  pd.DataFrame()
            for index, row in mergedf.iterrows():
                stats = get_boxscore_stats(row['Links'])
                vegas = getGameInfo(row['Links'])
                vegas_stats = stats.join(vegas.set_index('VegasLink'), on='GetBoxScoreURL')
                gameinfo = gameinfo.append(vegas_stats, ignore_index=True, sort=False)
            gameinfo = pd.merge(mergedf, gameinfo, left_on='Links', right_on='GetBoxScoreURL')
            if 'Date_x' in gameinfo.columns:
                gameinfo.rename(columns = {'Date_x':'Date'}, inplace=True)
            nflDF = nflDF.append(gameinfo,ignore_index=True)
            sleep(2)
        masterdf = masterdf.append(nflDF,ignore_index=True)
        sleep(2)
    return masterdf

In [7]:
master = get_master_df(team_list,2002,2019)
# df.to_csv(r'./export/dftocsv.csv', encoding='utf-8', header='true')

In [8]:
master.to_csv(r'./data/NFLMASTERDF.csv', header=True)
#S3 buck name . -  nflmasterdata
# df.to_csv(r'./export/dftocsv.csv', encoding='utf-8', header='true')